In [24]:
#!pip install pyhive

In [25]:
from os import path, getenv
from mlrun import new_project

#project_name = '-'.join(filter(None, ['getting-started-iris', getenv('V3IO_USERNAME', None)]))
project_name = "parquez"
project_path = path.abspath('./')
project = new_project(project_name, project_path)
project.save()
print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/parquez
Project name: parquez


In [26]:
%env PYTHONPATH=./

env: PYTHONPATH=./


In [27]:
project.set_function("parquez.py", 'parquezrun', kind='job', image='aviaigz/parquez')

In [28]:
from mlrun import run_local, mount_v3io

In [29]:
project.func('parquezrun').apply(mount_v3io())
project.func('parquezrun').set_env('PYTHONPATH', project_path)

In [30]:
project.func('parquezrun').run()

[mlrun] 2020-05-20 10:50:09,894 artifact path is not defined or is local, artifacts will not be visible in the UI
[mlrun] 2020-05-20 10:50:09,914 starting run parquezrun uid=d71ba720c0e94d619b72180556c63c13  -> http://10.194.96.98:8080
[mlrun] 2020-05-20 10:50:10,041 Job is running in the background, pod: parquezrun-4w28c
[mlrun] 2020-05-20 10:50:19,133 artifact path is not defined or is local, artifacts will not be visible in the UI
[mlrun] 2020-05-20 10:50:21,274 logging run results to: http://10.194.96.98:8080
[mlrun] 2020-05-20 10:50:21,306 Starting to Parquezzzzzzzz

[mlrun] 2020-05-20 10:50:21,633 exec error - usage: use runParquez --help for more information
Parquez: error: the following arguments are required: --view-name, --partition-by, --real-time-table-name

[mlrun] 2020-05-20 10:50:21,671 run executed, status=error
usage: use runParquez --help for more information
Parquez: error: the following arguments are required: --view-name, --partition-by, --real-time-table-name

run

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
parquez,...56c63c13,0,May 20 10:50:21,error,parquezrun,host=parquezrun-4w28ckind=jobowner=iguaziov3io_user=iguazio,,,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run d71ba720c0e94d619b72180556c63c13 --project parquez , !mlrun logs d71ba720c0e94d619b72180556c63c13 --project parquez
[mlrun] 2020-05-20 10:50:25,789 run executed, status=error
runtime error: usage: use runParquez --help for more information
Parquez: error: the following arguments are required: --view-name, --partition-by, --real-time-table-name



RunError: usage: use runParquez --help for more information
Parquez: error: the following arguments are required: --view-name, --partition-by, --real-time-table-name


In [ ]:
# Run the data-ingestion function locally in Jupyter Notebook
get_data_run = run_local(command= project.func('parquezrun'),workdir='./')